# Siamese Neural Networks for Face Recognition

Train  a neural network to identify if inut pictures belong to same person

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
import numpy as np
import pandas as pd
import cv2.cv2 as cv
import os
import random
import threading
import glob
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import matplotlib.pyplot as plt

if (torch.cuda.device_count()):
  print(torch.cuda.device_count())
  print(torch.cuda.get_device_name(0))

#Assign cuda GPU located at location '0' to a variable
  cuda0 = torch.device('cuda:0')
else:
  cuda0 = torch.device('cpu')

#### Define some utility functions

In [ ]:
#Function to read images from files and converting to torch tensors
def readImage(img_name,img_size = (224,224),plot=False):
    n = cv.imread(img_name)
    n = cv.resize(n, img_size, interpolation = cv.INTER_LINEAR)
    n_torch = np.transpose(n, (2, 0, 1))
    if plot:
      plt.imshow(cv.cvtColor(n, cv.COLOR_BGR2RGB))
      plt.show()

    img_torch = torch.tensor(n_torch)
    # img_torch = img_torch.float()
    # img_torch = img_torch.reshape((1,n_torch.shape[0],n_torch.shape[1],n_torch.shape[2]))
    # img_torch = img_torch/255

    return n,img_torch

#Function to generate pairs of images
def generate_train_image_pairs(Names_list, Names_dict,pos_pairs,neg_pairs):    
    pair_images = []
    pair_labels = []
    N_names = len(Names_list)
    for Name,index in zip(Names_list,tqdm(range(1,N_names))):
        file_names = Names_dict[Name]

        image = file_names[0]
        print("**********************")
        print("imag base ", image)
        _,base_img_torch = readImage(image,img_size,plot=False)
        
        #----------------#
        #Get positive pairs (pairs from same person)

        pos_indices = np.random.choice(len(file_names),pos_pairs)
        for p in pos_indices:
          pos_image = file_names[p]
          print("pos image ", pos_image)
          _,pos_img_torch = readImage(pos_image,img_size,plot=False)

          pair_images.append((base_img_torch, pos_img_torch))
          pair_labels.append(torch.ones(1,1))

          #Augment positive pairs.
          #This part just incrases the datasize with augmented pictures
          for n_aug in range(3):
            Aug_img_base = augmenter(base_img_torch)
            Aug_img_pos = augmenter(pos_img_torch)
            pair_images.append((Aug_img_base, Aug_img_pos))
            pair_labels.append(torch.ones(1,1))

            # plt.imshow(cv.cvtColor(np.transpose(Aug_img_base.numpy(),(1,2,0)), cv.COLOR_BGR2RGB))
            # plt.imshow(cv.cvtColor(np.transpose(Aug_img_pos.numpy(),(1,2,0)), cv.COLOR_BGR2RGB))
            # plt.show()
            
            
        #----------------#
        #Get negative pairs (pairs from different people)

        neg_indices = np.where(Names_list != Name)[0]
        neg_indices = np.random.choice(neg_indices,neg_pairs)

        for n in neg_indices:
          Name_neg = Names_list[n]
          file_names = Names_dict[Name_neg]

          neg_image = file_names[np.random.choice(len(file_names))]
          print("neg  image ", neg_image)
          _,neg_img_torch = readImage(neg_image,img_size,plot=False)

          pair_images.append((base_img_torch, neg_img_torch))
          pair_labels.append(0*torch.ones(1,1))

         #AUgment Images
          for n_aug in range(3):
            Aug_img_base = augmenter(base_img_torch)
            Aug_img_neg = augmenter(neg_img_torch)
            pair_images.append((Aug_img_base, Aug_img_neg))
            pair_labels.append(0*torch.ones(1,1))
          
    return pair_images, pair_labels

#Function to divide the data into batches
def GetBatches(pair_images,pair_labels,image_size,n_batches):
  #**********To Do
  batch_size = len(pair_images)//n_batches
  print("batch size ", batch_size)

  Batch_Images_1,Batch_Images_2,Batch_Labels = [], [],[]
  for i in range(0,len(pair_images),batch_size):
    images = pair_images[i:i+batch_size]
    labels = pair_labels[i:i+batch_size]

    Images_1 = torch.empty((0,3,image_size[0],image_size[1]))
    Images_2 = torch.empty((0,3,image_size[0],image_size[1]))
    Labels = torch.empty((0,1))

    for image_pair,label in zip(images,labels):
      image_pair_1 = image_pair[0].reshape(1,3,image_size[0],image_size[1])
      image_pair_2 = image_pair[1].reshape(1,3,image_size[0],image_size[1])

      Images_1 = torch.vstack((Images_1,image_pair_1))
      Images_2 = torch.vstack((Images_2,image_pair_2))
      Labels = torch.vstack((Labels,label.reshape(-1,1)))

    Images_1 = Images_1.to(cuda0)
    Images_2 = Images_2.to(cuda0)
    Labels = Labels.to(cuda0)

    # print("Batch Images shape ", Images_1.shape, " Labels",Labels.shape)

    Batch_Images_1.append(Images_1)
    Batch_Images_2.append(Images_2)
    Batch_Labels.append(Labels)

  return Batch_Images_1,Batch_Images_2,Batch_Labels

#### Get the data from the folder

In [ ]:
folder_root = "pubfig83/" 

img_size = (90,90)
augmenter = T.RandAugment()

np.random.seed(seed=42)

#Get persons' names and images
Names = dict() #dictionary of names and all files in subfolder
subfolders = []
el_counts = []
Images_names = []
for x in os.walk(folder_root):
  name = x[0].split("/")
  filenames = glob.glob(x[0]+"/*.jpg")
  if len(filenames):
    Names[name[1]] = filenames
    subfolders.append((x[0],filenames))
    for filename in filenames:
      Images_names.append(filename )

Names_keys = list(Names.keys())

#Generate the pairs of images
#pos_pairs and neg_pairs set the number of pairs to generate for each person
pair_images,pair_labels = generate_train_image_pairs(np.array(Names_keys),Names,pos_pairs=5,neg_pairs=5)


#Split the data into training and test
pair_images_train, pair_images_test, pair_labels_train, pair_labels_test = train_test_split(pair_images, pair_labels, test_size=0.15, random_state=42)
print("Train data size", len(pair_images_train))
print("Test data size", len(pair_images_test))

#Split data into batches for training
n_batches_train = 10
Batch_Im_1,Batch_Im_2,Batch_Labels = GetBatches(pair_images_train,pair_labels_train,img_size,n_batches_train)
Data_train = list(zip(Batch_Im_1,Batch_Im_2,Batch_Labels))

#Single batch used for testing
n_batches_test = 1
Batch_Im_1,Batch_Im_2,Batch_Labels = GetBatches(pair_images_test,pair_labels_test,img_size,n_batches_test)
Data_test = list(zip(Batch_Im_1,Batch_Im_2,Batch_Labels))

#### Define Network Architecture

In [ ]:
# Convolutional layer block with maxpooling
class ConvNetBlock(nn.Module):
  def __init__(self,in_channel,out_channel,kernel_size,pool=True):
    super(ConvNetBlock,self).__init__()

    self.pool = pool
    self.kernel_size = kernel_size
    
    self.batchNorm = torch.nn.BatchNorm2d(in_channel)

    self.conv = torch.nn.Conv2d(in_channel, out_channel,kernel_size, stride=1)
    self.act = nn.ReLU()
    if pool:
      self.maxpool = torch.nn.MaxPool2d(2)

  def forward(self,x):
    x = self.batchNorm(x)
    x = self.conv(x)
    x = self.act(x)

    if self.pool:
      x = self.maxpool(x)

    return x

#returns output figure size
  def getImageSize(self,image_size):
    h_in = image_size[0]
    w_in = image_size[1]

    h_out = int(h_in-self.kernel_size+1)
    w_out = int(w_in-self.kernel_size+1)

    if self.pool:
      h_out = int((h_out-2)/2+1)
      w_out = int((w_out-2)/2+1)

    return (h_out,w_out)

# Neywork for fetaure extraction
class FeatureExtractNet(nn.Module):
  def __init__(self,image_shape):
    super(FeatureExtractNet,self).__init__()

    self.conv = nn.ModuleList()
    self.FC = nn.ModuleList()

    self.conv.append(ConvNetBlock(3,16,5,True))
    image_shape = self.conv[0].getImageSize(image_shape)

    self.conv.append(ConvNetBlock(16,32,5,True))
    image_shape = self.conv[1].getImageSize(image_shape)

    self.conv.append(ConvNetBlock(32,64,3,True))
    image_shape = self.conv[2].getImageSize(image_shape)

    self.conv.append(ConvNetBlock(64,128,3,False))
    image_shape = self.conv[3].getImageSize(image_shape)
    
    print("final image shape conv: ",image_shape)

    in_dim = image_shape[0]*image_shape[1]*128
    
    # Fully connected layers
    self.FC.append(torch.nn.Linear(in_dim,1000))
    self.sigm = torch.nn.Sigmoid()
    self.Out = torch.nn.Linear(1000,100)

  def forward(self,x):

    for conv in self.conv:
      x = conv(x)

    x = x.reshape(x.shape[0],-1)

    for fc in self.FC:
      x = fc(x)
      x = self.sigm(x)

    out = self.Out(x)

    return out

#This block computes the distance between feature vectors
class DiffNet(nn.Module):
  def __init__(self,image_shape):
    super(DiffNet,self).__init__()

    self.ExtractNet = FeatureExtractNet(image_shape)

    self.diff = torch.nn.Linear(1,1)
    self.sigm = torch.nn.Sigmoid()

    self.EuDist = torch.nn.PairwiseDistance(p=2)

    #x_1 and x_2 are input pictures
  def forward(self,x_1,x_2):

    feat_1 = self.ExtractNet(x_1) #predicted features for input 1
    feat_2 = self.ExtractNet(x_2) #predicted features for input 2
    
    #Euclidean distance reshaped to be one-dimensional
    diff = self.EuDist(feat_1,feat_2)/feat_1.shape[1]
    diff = diff.reshape(-1,1) 

    
    out = self.diff(diff) # computes out = w*d+b from linear layer
    out = self.sigm(out) # probability distribution

    return out,diff #probability distribution and feature distance

#### Define the Loss and Training

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

class ContrastiveLoss(torch.nn.Module):

    def __init__(self, margin=1):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, euclidean_distance, label):
        # euclidean_distance = F.pairwise_distance(output1, output2)
        pos = (label) * torch.pow(euclidean_distance, 2)
        neg = (1-label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        loss_contrastive = torch.mean( pos + neg )
        return loss_contrastive

def TrainNet(Net,Data_train,Data_test,lr=1e-03,Epochs = 100):

    BCE_loss_function = nn.BCELoss()
    C_loss_function = ContrastiveLoss()
    optimizer = torch.optim.Adam(Net.parameters(), lr=lr)  

    N_batches = len(Data_train)

    for epoch in tqdm(range(1,Epochs)):
      loss_batch = 0
      accuracy = 0
      n_batch = 0
      str = ""
      for x1,x2,y in Data_train:
        prob_pred,diff = Net(x1,x2)
          
        BCE_loss = BCE_loss_function(prob_pred,y)
        loss_Contr = C_loss_function(diff,y)

        loss = loss_Contr+0.1*BCE_loss

        loss_batch = loss_batch + loss.item()

        y_pred = torch.where(prob_pred >= 0.5,1,0)

        accuracy = accuracy+(y_pred==y).sum().item()/y.shape[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        n_batch = n_batch+1

      loss_batch = loss_batch/N_batches
      accuracy_train = accuracy/len(Data_train)
      

      #Validation accuracy:
      loss_test = 0
      accuracy = 0
      for x1,x2,y in Data_test:
        with torch.no_grad(): 
          Net_eval = Net.eval()       
          prob_pred,diff = Net_eval(x1,x2)

        # loss = loss_function(y_pred,y)
        loss = C_loss_function(diff,y)
        # loss = ContrastiveLoss(diff,y)

        y_pred = torch.where(prob_pred >= 0.5,1,0)
        accuracy = accuracy+(y_pred==y).sum().item()/y.shape[0]

        loss_test = loss_test+loss.item()

      loss_test = loss_test/len(Data_test)
      accuracy = accuracy/len(Data_test)

      print("*********")
      print("epoch: ",epoch," loss train: ",loss_batch," accuracy train ",accuracy_train," loss test",loss_test," accuracy test",accuracy)

    return Net

#### Train the Network

In [ ]:
Net = DiffNet(img_size)
Net.to(cuda0)

Net = TrainNet(Net,Data_train,Data_test,lr=1e-03,Epochs = 150)

# torch.save(Net, "SiameseNet_FaceRec.pth")

#### Test on Training and Testing Data

In [ ]:
#test results on train data
Images_1,Images_2,Labels = Data_train[1]
idxs = np.random.choice(Images_1.shape[0], size=10)

# Net = torch.load("SiameseNet_FaceRec.pth")

# Net = Net.cpu()
Net_eval = Net.eval() 

accuracy = 0
for idx in idxs:
  print("************************************")
  with torch.no_grad():
    Img_1 = Images_1[idx]
    Img_2 = Images_2[idx]
    label = Labels[idx]  
    
    proba_pred,diff = Net_eval(Img_1.reshape(1,Img_1.shape[0],Img_1.shape[1],Img_1.shape[2]),Img_2.reshape(1,Img_1.shape[0],Img_1.shape[1],Img_1.shape[2]))

  Img_1 = Img_1.cpu().numpy()
  Img_2 = Img_2.cpu().numpy()
  label = label.cpu().numpy()
  proba_pred = proba_pred.cpu().numpy()[0]

  Img_1 = np.transpose(Img_1, (1, 2, 0))
  Img_2 = np.transpose(Img_2, (1, 2, 0))

  Img_1 =  Img_1.astype(np.uint8)
  Img_2 =  Img_2.astype(np.uint8)

  print(Img_1.shape)

  fig, (ax1, ax2) = plt.subplots(1,2)
  ax1.imshow(cv.cvtColor(Img_1, cv.COLOR_BGR2RGB))
  ax2.imshow(cv.cvtColor(Img_2, cv.COLOR_BGR2RGB))

  plt.show()
  print(" similarity ",proba_pred, " truth", label)

  if proba_pred >= 0.5:
    label_pred = 1
  else:
    label_pred = 0

  accuracy = accuracy+(label_pred==label).sum()

accuracy = accuracy/len(idxs)
print("Average accurcay", accuracy)

#-------------------------------------------------------------#

#test results on train data
Images_1,Images_2,Labels = Data_test[0]
idxs = np.random.choice(Images_1.shape[0], size=min(Images_1.shape[0],200))

# Net = Net.cpu()
Net_eval = Net.eval() 

accuracy = 0
for idx in idxs:
  print("************************************")
  with torch.no_grad():
    Img_1 = Images_1[idx]
    Img_2 = Images_2[idx]
    label = Labels[idx]  
    
    proba_pred,diff = Net_eval(Img_1.reshape(1,Img_1.shape[0],Img_1.shape[1],Img_1.shape[2]),Img_2.reshape(1,Img_1.shape[0],Img_1.shape[1],Img_1.shape[2]))

  Img_1 = Img_1.cpu().numpy()
  Img_2 = Img_2.cpu().numpy()
  label = label.cpu().numpy()
  proba_pred = proba_pred.cpu().numpy()
  proba_pred = proba_pred[0]

  Img_1 = np.transpose(Img_1, (1, 2, 0))
  Img_2 = np.transpose(Img_2, (1, 2, 0))

  Img_1 =  Img_1.astype(np.uint8)
  Img_2 =  Img_2.astype(np.uint8)

  print(Img_1.shape)

  fig, (ax1, ax2) = plt.subplots(1,2)
  ax1.imshow(cv.cvtColor(Img_1, cv.COLOR_BGR2RGB))
  ax2.imshow(cv.cvtColor(Img_2, cv.COLOR_BGR2RGB))

  plt.show()
  print(" similarity ",proba_pred, " truth", label)

  if proba_pred >= 0.5:
    label_pred = 1
  else:
    label_pred = 0

  accuracy = accuracy+(label_pred==label).sum()

accuracy = accuracy/len(idxs)
print("Average accurcay", accuracy)